In [1]:
!pip install -U interpret

     |████████████████████████████████| 3.8MB 2.8MB/s 
     |████████████████████████████████| 440kB 46.0MB/s 
     |████████████████████████████████| 757kB 44.9MB/s 
     |████████████████████████████████| 778kB 43.2MB/s 
     |████████████████████████████████| 276kB 46.0MB/s 
     |████████████████████████████████| 3.4MB 33.6MB/s 
     |████████████████████████████████| 747kB 45.6MB/s 
     |████████████████████████████████| 4.2MB 22.9MB/s 
     |████████████████████████████████| 921kB 40.9MB/s 
     |████████████████████████████████| 40kB 19.8MB/s 
     |████████████████████████████████| 542kB 47.1MB/s 
     |████████████████████████████████| 41.2MB 642kB/s 
     |████████████████████████████████| 235kB 41.0MB/s 
     |████████████████████████████████| 2.0MB 21.3MB/s 
     |████████████████████████████████| 235kB 46.1MB/s 
     |████████████████████████████████| 122kB 45.9MB/s 
     |████████████████████████████████| 337kB 50.0MB/s 
     |████████████████████████████████| 471kB 45.9

In [0]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Convertimos la etiqueta en 0 y 1

# Transformamos las variables categóricas para utilizar modelos de sklearn
X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)

seed = 1  
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=seed)

In [3]:
#Entrenando el modelo

from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# Podemos incluir prepocesado, no solo el clasificador dentro de nuestro pipeline

pca = PCA()
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

blackbox_model = Pipeline([('pca', pca), ('rf', rf)])
blackbox_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('rf',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=-1,
                                        oob_score=False, random_state=None,
                                        verbose=0, wa

In [0]:
#Viendo el rendimiento

from interpret import show
from interpret.perf import ROC

blackbox_perf = ROC(blackbox_model.predict_proba).explain_perf(X_test, y_test, name='Blackbox')
show(blackbox_perf)

In [0]:
#Explicaciones locales

from interpret.blackbox import LimeTabular
from interpret import show

#Blackbox explainers need a predict function, and optionally a dataset
lime = LimeTabular(predict_fn=blackbox_model.predict_proba, data=X_train, random_state=1)

#Pick the instances to explain, optionally pass in labels if you have them
lime_local = lime.explain_local(X_test[:5], y_test[:5], name='LIME')

show(lime_local)

In [0]:
from interpret.blackbox import ShapKernel
import numpy as np

background_val = np.median(X_train, axis=0).reshape(1, -1)
shap = ShapKernel(predict_fn=blackbox_model.predict_proba, data=background_val, feature_names=feature_names)
shap_local = shap.explain_local(X_test[:5], y_test[:5], name='SHAP')
show(shap_local)

In [0]:
#Explicaciones Globales
from interpret.blackbox import MorrisSensitivity

sensitivity = MorrisSensitivity(predict_fn=blackbox_model.predict_proba, data=X_train)
sensitivity_global = sensitivity.explain_global(name="Global Sensitivity")

show(sensitivity_global)


In [0]:
from interpret.blackbox import PartialDependence

pdp = PartialDependence(predict_fn=blackbox_model.predict_proba, data=X_train)
pdp_global = pdp.explain_global(name='Partial Dependence')

show(pdp_global)

In [0]:
show([blackbox_perf, lime_local, shap_local, sensitivity_global, pdp_global])